In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import datasets
from torch_mas.ciel import Ciel

## Loading the iris dataset

In [10]:
data = datasets.load_iris()

## Splitting the dataset intro train and test

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=0)

## Learning with Context

In [14]:
from torch_mas.ciel import Ciel

re = Ciel(
    4, 
    1, 
    R=0.8,
    imprecise_th=0.5,
    bad_th=0.0015,
    alpha=0.5,
    memory_length=3,
    n_epochs=5
    )

## Training the agents

In [15]:
import time
t = time.time()
re.fit(X_train, y_train)
tt = time.time() - t
print(f"Total training time: {tt}s")

print("Number of agents created:", re.estimator.agents.n_agents)

Total training time: 0.8000006675720215s
Number of agents created: 41


## Computing performance

In [16]:
y_pred = re.predict(X_test)
print(classification_report(y_test, y_pred))

<class 'numpy.ndarray'>


# Hyperparameter optimisation

In [18]:
dic = {
    "R" : {0,1},
    "bad_th" : {0,1},
    "alpha" : {0,1},
}
init = 0.1
best = math.inf
bestdic = []
for R in range(1,10):
    for b in range(1,10):
        for a in range(1,10):
            model = Head(
            feature_size, 
            target_size, 
            R=R*init,
            imprecise_th=0.5,
            bad_th=b*init,
            alpha=a*init,
            memory_length=3,
            n_epochs=5
            )
            dataset = DataBuffer(X_train,y_train)
            t = time.time()
            model.fit(dataset)
            y_pred = model.predict(torch.from_numpy(X_test).float()).round()
            
            differences = torch.ne(y_pred.squeeze(), y_test)
            num_differences = differences.sum().item()
            #print("Parameters : ",R*init,":",b*init,":",a*init," Score:",num_differences)
            if(num_differences < best):
                bestdic = [{R,b,a}]
                best = num_differences
            elif num_differences == best:
                bestdic.append({R,b,a})
print("Best parameters", bestdic, " with Score", best)

Best parameters [{8, 1}, {8, 2}, {8, 9, 2}, {8, 3}, {8, 9, 6}, {8, 5}, {8, 9, 2}, {8, 9, 4}, {8, 9, 6}, {9, 1}, {9, 2, 4}, {9, 3, 7}, {9, 4, 1}, {9, 3, 4}, {9, 4}, {8, 9, 4}, {9, 2, 5}, {9, 4, 5}, {9, 6, 1}, {9, 4, 6}, {9, 7}, {8, 9, 4}, {8, 9}]  with Score 0
